In [ ]:
import tensorflow as tf


In [ ]:
model_path = "results/samuel/MobileNetV2_MC_KerasModel/model.tf"
model = tf.keras.models.load_model(model_path)
model.input.set_shape((1,256,256,3))
model.summary()

In [10]:
from custom import *
from create_model import generate_model
import tensorflow as tf

model = generate_model((256,256,3))
model.input.set_shape((1,256,256,3))
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(1, 256, 256, 3)]   0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 128, 128, 32) 864         input_4[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 128, 128, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 128, 128, 32) 0           bn_Conv1[0][0]                   
____________________________________________________________________________________________

In [11]:
#converter = tf.lite.TFLiteConverter.from_saved_model(model_path)
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.experimental_new_converter = True
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()


# Save the model.
with open('results/coral_chip/MobileNetV2/model.tflite', 'wb') as f:
  f.write(tflite_model)



In [ ]:
from matplotlib import image
import numpy as np

with open("plots/predictions/encoder_sC/img_0.npy", "rb") as f:
    img = np.load(f)

image.imsave("test.png", img)